In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import optiver2023 

In [ ]:
# 加载训练数据集
train_data = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

# 删除认为对模型预测没有帮助的列
X = train_data.drop(['row_id', 'time_id', 'date_id', 'stock_id', 'target'], axis=1)
y = train_data['target']

# 处理NaN值，这里使用中位数进行填充
X.fillna(X.median(), inplace=True)
y.fillna(y.median(), inplace=True)

# 分割数据集，用于训练和验证
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# 数据标准化和模型创建的管道
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])

# 设置岭回归的参数范围
param_grid = {
    'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# 创建网格搜索对象
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 找到最佳模型
best_model = grid_search.best_estimator_

In [ ]:
# 使用比赛API获取隐藏测试集
env = optiver2023.make_env() 
iter_test = env.iter_test()

for (test_df, revealed_targets, sample_prediction_df) in iter_test:
    # 删除在训练集中未使用的特征
    features_to_use = X_train.columns  # 训练集中使用的特征列
    X_test = test_df[features_to_use]  # 确保只使用这些特征
    
    # 填充NaN值，确保没有缺失值
    X_test.fillna(X_train.median(), inplace=True)
    
    # 使用最佳模型进行预测
    y_pred_test = best_model.predict(X_test)
    
    # 将预测结果填充到sample_prediction_df中
    sample_prediction_df['target'] = y_pred_test
    
    # 使用env.predict提交预测结果
    env.predict(sample_prediction_df)